In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import torch 
import pytorch_lightning as pl
import pandas as pd

from utils.config import TRAIN_LABELS_FILE, TEST_FOLDER, TRAIN_FOLDER

In [3]:
df = pd.read_csv(TRAIN_LABELS_FILE)

In [13]:
count = df['label'].value_counts()
count

0    27113
1    14048
Name: label, dtype: int64

In [14]:
count[0]/count[1]

1.9300256264236901

In [8]:
len(os.listdir(TEST_FOLDER))

17794

# Training

In [1]:
import torch

from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint

from utils.datasets import FakeVideoDataModule
import torchvision.models as models

Global seed set to 42


In [2]:
# base_model = models.video.r3d_18(pretrained=True, progress=False)
# base_model = torch.hub.load('facebookresearch/pytorchvideo', 'x3d_s', pretrained=False)
# base_model = models.efficientnet_b7(pretrained=True)

# classifier = FakeVideoDetector(model=base_model, lr=1e-2)

In [14]:
from models.efficientnet import PretrainedEfficientNet, get_effnet_transform

classifier = PretrainedEfficientNet(lr=1e-3)

In [15]:

datamodule = FakeVideoDataModule(num_workers=2, batch_size=64, transforms=get_effnet_transform())

In [16]:
from pytorch_lightning.callbacks import BackboneFinetuning

callbacks = [
            EarlyStopping(monitor="AUROC", mode="max", patience=20),
            BackboneFinetuning(10, should_align=True),
            # ModelCheckpoint(
            #     dirpath=f'checkpoints',
            #     filename='{epoch}--{AUROC:.3f}', monitor="AUROC", mode="max",
            # ),
        ]

trainer = Trainer(
    gpus=1,
    callbacks=callbacks,
    log_every_n_steps=5,
    precision=16,
    # gradient_clip_val=0.01,
    # deterministic=True,
)


Using native 16bit precision.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [ ]:
trainer.fit(classifier, datamodule)

# Misc

In [9]:
import matplotlib.pyplot as plt

In [10]:
batch['video'].shape

torch.Size([4, 3, 13, 182, 182])

In [ ]:
for i in range(13):
    im = batch['video'][0, :, i, :]
    plt.imshow(im.permute(1,2,0))
    plt.show()

In [12]:
batch['video_name']

['hzeawunfbwbwjwwoiemmuirtolucysmo.mp4',
 'tlfrfdvrsmmzlanfrpuuxvmtiklbyqvv.mp4',
 'hfbtqqpivyjpzuztyeegsldahohwjaal.mp4',
 'wwbsivgzzzzgblfzscaldiezstivnozd.mp4']

In [13]:
batch.keys()

dict_keys(['video', 'video_name', 'video_index', 'clip_index', 'aug_index', 'label'])